[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_latent_upscale.ipynb)

In [ ]:
!pip install -q git+https://github.com/huggingface/diffusers.git 
!pip install -q transformers accelerate safetensors --pre xformers triton

from diffusers import StableDiffusionLatentUpscalePipeline, StableDiffusionPipeline
import torch

pipeline = StableDiffusionPipeline.from_pretrained("ckpt/anything-v4.5-vae-swapped", torch_dtype=torch.float16, safety_checker=None).to("cuda")
pipeline.enable_xformers_memory_efficient_attention()
upscaler = StableDiffusionLatentUpscalePipeline.from_pretrained("stabilityai/sd-x2-latent-upscaler", torch_dtype=torch.float16).to("cuda")
upscaler.enable_xformers_memory_efficient_attention()

In [ ]:
prompt = "mermaid"
generator = torch.manual_seed(420)

latents = pipeline(prompt, generator=generator, output_type="latent").images
high = upscaler(prompt, generator=generator, image=latents, num_inference_steps=50, guidance_scale=0).images[0]

with torch.no_grad():
    low = pipeline.decode_latents(latents)[0]
low = pipeline.numpy_to_pil(low)[0]

In [ ]:
display(low)

In [ ]:
display(high)